In [3]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
from keras.layers import LSTM, Dropout, Dense
from keras.layers import SimpleRNN
from keras.layers import GRU
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
# imort Sequential
from keras.models import Sequential

**Read data**


In [4]:
import pandas as pd
df = pd.read_csv("/Users/rahuljauhari/Desktop/research-runoff/Data/merged_imd.csv")
df.drop(columns=['Unnamed: 0'], inplace=True)
df = df.iloc[:, :157]

df['DateTime'] = pd.to_datetime(df['DateTime'])
df.set_index('DateTime', inplace=True)
monthly_mean = df.resample('M').mean()
print(monthly_mean.shape)

df_actual = pd.read_excel("/Users/rahuljauhari/Desktop/research-runoff/Data/Calibrated and Validated.xlsx")
# select last column
observed_runnoff = df_actual['observed']
# observed_runnoff.head()
print(observed_runnoff.shape)

(468, 156)
(468,)


In [5]:
from scipy.stats import zscore


def func(name):
    x = 0
    y = 0
    inv = 0
    if name == 'zscore':
        x_norm = zscore(monthly_mean)
        y_norm = zscore(observed_runnoff)
        x_norm[x_norm > 3] = 2.8
        x_norm[x_norm < -3] = -2.8
        y_norm[y_norm > 3] = 2.8
        y_norm[y_norm < -3] = -2.8
        x = x_norm
        y = y_norm
    if name == 'StandardScaler':
        scaler = StandardScaler()
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1, 1))
        x_scaled[x_scaled > 3] = 2.8
        x_scaled[x_scaled < -3] = -2.8
        y_scaled[y_scaled > 3] = 2.8
        y_scaled[y_scaled < -3] = -2.8
        x = x_scaled
        y = y_scaled
        inv = scaler

    if name == 'MinMaxScaler':
        scaler = MinMaxScaler(feature_range=(0, 1))
        x_scaled = scaler.fit_transform(monthly_mean)
        y_scaled = scaler.fit_transform(observed_runnoff.values.reshape(-1, 1))
        x = x_scaled
        y = y_scaled
        inv = scaler
    return x, y, inv


**Metric**

In [6]:
from sklearn.model_selection import train_test_split
# adam optimizer import
from keras.optimizers import Adam, RMSprop, SGD
# x, y, inv = func('MinMaxScaler')
# x_train, x_test, y_train, y_test = train_test_split(
#     x, y, test_size=0.3, shuffle=False)
# # validation_data 10% of train data
# x_train, x_val, y_train, y_val = train_test_split(
#     x_train, y_train, test_size=0.1, shuffle=False)


In [7]:
activation = [ 'linear']
optimizer = ['adam']
preprocess = ['StandardScaler']

In [9]:
activation = [ 'tanh']
optimizer = ['adam']
preprocess = ['StandardScaler']

In [10]:
from sklearn.metrics import r2_score
for act in activation:
    for opt in optimizer:
        for pre in preprocess:
            x,y,inv_scaler= func(pre)
            X_train, X_test,y_train,y_test = train_test_split(x,y,test_size=0.2,shuffle=False)
            model = keras.Sequential()
            model.add(GRU(64,return_sequences=True, activation=act, input_shape=(X_train.shape[1],1)))
            model.add(Dense(64))
            model.add(Dropout(0.5))
            model.add(LSTM(64, activation=act))
            model.add(Dense(64))
            model.add(Dropout(0.5))
            model.add(layers.Dense(1))
            model.compile(optimizer=opt, loss='mse')
            model.fit(X_train, y_train, batch_size=10, epochs=100,shuffle=False, use_multiprocessing=True,verbose=0,validation_split=0.1)
            y_pred = model.predict(X_test)
            y_pred_train = model.predict(X_train)
            try:
                y= np.concatenate((y_pred_train,y_pred),axis=0)
                # inverse transform
                y_inv = inv_scaler.inverse_transform(y)
                # to csv
                pd.DataFrame(y_inv).to_csv('/Users/rahuljauhari/Desktop/research runoff/results1/imd_gru_lstm_0.2.csv',mode='a',header=True)
            except Exception as e:
                print(e)


In [ ]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred_train_inv = inv_scaler.inverse_transform(y_pred_train)
y_pred_test_inv = inv_scaler.inverse_transform(y_pred_test)
y_train__inv = observed_runnoff[:len(y_pred_train_inv)]
y_test__inv = observed_runnoff[len(y_pred_train_inv):]
print("KGE train: ", round(kge1(y_train__inv, y_pred_train_inv),4))
print("KGE test: ", round(kge1(y_test__inv, y_pred_test_inv),4))



In [ ]:
# # keras tuner for gru lstm
# from keras import backend as K
# from sklearn.metrics import r2_score
# import keras_tuner as kt
# from keras_tuner.tuners import RandomSearch
# from keras import regularizers
# from keras_tuner.engine.hyperparameters import HyperParameters


# def build_model(hp):
#     model = keras.Sequential()
#     # for i in range(hp.Int('n_layers', 1, 2)):
#     model.add(layers.GRU(units=hp.Int('units', min_value=32, max_value=512, step=32), activation=hp.Choice(
#         'act_' , values=['relu', 'sigmoid', 'linear', 'tanh']),  return_sequences=True, input_shape=(x_train.shape[1], 1)))
#     model.add(layers.Dense(1, activation=hp.Choice(
#         'act_' , values=['relu', 'sigmoid', 'linear', 'tanh'])))
#     model.add(Dropout(hp.Float('dropout_1', 0, 0.5, step=0.1, default=0.2)))
#     # for i in range(hp.Int('n_layers', 1, 2)):
#     model.add(layers.LSTM(units=hp.Int('units', min_value=32, max_value=512, step=32), activation=hp.Choice(
#         'act_' , values=['relu', 'sigmoid', 'linear', 'tanh']), return_sequences=False))
#     model.add(Dropout(hp.Float('dropout_1', 0, 0.5, step=0.1, default=0.2)))
#     model.add(layers.Dense(1, activation=hp.Choice('act_' , values=[
#         'relu', 'sigmoid', 'linear', 'tanh']), kernel_regularizer=regularizers.l2(hp.Float('l2', 0, 0.5, step=0.1, default=0.2))))
#     hp_lr = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
#     hp_optimizer = hp.Choice('optimizer', values=['sgd', 'rmsprop', 'adam'])
#     if hp_optimizer == 'sgd':
#         optimizer = SGD(learning_rate=hp_lr)
#     elif hp_optimizer == 'rmsprop':
#         optimizer = RMSprop(learning_rate=hp_lr)
#     else:
#         optimizer = Adam(learning_rate=hp_lr)
#     model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])
#     return model


# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
# tuner = RandomSearch(build_model, objective='mse', max_trials=10,
#                      executions_per_trial=1, directory='project', project_name='GRU_LSTM')
# tuner.search(x_train, y_train, epochs=100, verbose=1,
#              validation_data=(x_val, y_val), callbacks=[early_stopping])
# model = tuner.get_models(num_models=1)[0]

In [ ]:
# from sklearn.metrics import r2_score
# print("train ",r2_score(y_train, model.predict(x_train)))
# print("val ",r2_score(y_val, model.predict(x_val)))
# print("test ",r2_score(y_test, model.predict(x_test)))
# model.summary()
# # optimizer used in best model
# print(model.optimizer.get_config())
# # activation used in best model
# model.layers[0].get_config()['activation']

In [ ]:
# # droupout
# model.layers[2].get_config()['rate']

In [ ]:
# # regularizer
# model.layers[5].get_config()['kernel_regularizer']